In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model
import scipy.special as sp

from Source import AWGN

Using TensorFlow backend.


In [2]:
n_power=7
CodeRate=0.5
N_length=int(2**n_power)
msg_length=int(CodeRate*N_length)
frozen_length=N_length-msg_length
codeNum =10**6
test_codeNum = 10**2

EbNo_Start=1.0  # in dB
EbNo_delta=0.5 # in dB
EbNo_Numbers=10 
EbNo=np.arange(EbNo_Start,EbNo_Start+EbNo_Numbers*EbNo_delta,EbNo_delta)
ebno=10.**(EbNo/10.)          # dB to real value 변환
BER_BPSK_Uncoded=0.5*sp.erfc(np.sqrt(ebno))      # BPSK BER 공식

In [3]:
G_All=AWGN.AllGeneratorMatrixPolarCode(n_power) ###생성행렬생성
(Z_AWGN,Z_decending_index_AWGN)=AWGN.BhanttacharyyaParameterAWGN(N_length, n_power, CodeRate)
SelectedBitsPositions = Z_decending_index_AWGN[frozen_length:]
SelectedBitsPositions.sort()
FrozenBitsPositions = Z_decending_index_AWGN[:frozen_length]
FrozenBitsPositions.sort()

In [4]:
model = load_model('Model/model_AWGN_p6_2.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
# msg=np.random.randint(0,2,size=(codeNum, msg_length))

In [6]:
# codeword, uncodedbits = AWGN.Encoding(msg, codeNum, N_length, n_power, msg_length, SelectedBitsPositions, G_All[n_power])
# ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate, codeNum, N_length, codeword)

In [7]:
# test_msg=np.random.randint(0,2,size=(test_codeNum, msg_length))
# test_msg, test_codeword, test_uncodedbits = AWGN.Encoding(test_msg, test_codeNum, N_length, n_power, msg_length, SelectedBitsPositions, 
#                                         G_All[n_power]) 
# test_ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate, test_codeNum, N_length, test_codeword)

In [8]:
msg=np.random.randint(0,2,size=(codeNum, msg_length))
msg, codeword, uncodedbits = AWGN.Encoding(msg, codeNum, N_length, n_power, msg_length, SelectedBitsPositions, G_All[n_power])

In [11]:
for i in range(10):
    
    ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate, codeNum, N_length, codeword)
    test_ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate, codeNum, N_length, codeword)
    
    model.fit(ReceivedAWGN, msg, 
          epochs=25,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_ReceivedAWGN, msg), verbose=0)
    
    if i%1 == 0:
        
        
        loss = model.evaluate(test_ReceivedAWGN, msg, verbose=0)
        print(i, loss)
        
        model.save('Model/model_AWGN_p6_2.h5')

0 [0.603569404888153, 0.6121640205383301]
1 [0.6034802080097198, 0.6120784878730774]
2 [0.6035430443229676, 0.612330973148346]
3 [0.6032143498401642, 0.6124252080917358]
4 [0.6035635721340179, 0.6124635338783264]
5 [0.6035735955371857, 0.612330436706543]
6 [0.603662318447113, 0.6122427582740784]
7 [0.6038317076358796, 0.6124486923217773]
8 [0.6036110966567994, 0.6125379800796509]
9 [0.6034373566856385, 0.612854540348053]


In [12]:
loss = model.evaluate(test_ReceivedAWGN, msg, verbose=0)
print(i, loss)
        
model.save('Model/model_AWGN_p6_2.h5')

9 [0.6034373566856385, 0.612854540348053]
